In [1]:
import gtsam
import numpy as np
from gtsam.gtsam import (Cal3_S2, DoglegOptimizer,
                         GenericProjectionFactorCal3_S2, NonlinearFactorGraph,
                         Point3, Pose3, Point2, PriorFactorPoint3, PriorFactorPose3,
                         Rot3, SimpleCamera, Values)

from utils import get_matches_and_e, load_image

In [2]:
def symbol(name: str, index: int) -> int:
    """ helper for creating a symbol without explicitly casting 'name' from str to int """
    return gtsam.symbol(ord(name), index)

In [3]:
def get_camera_calibration(fx, fy, s, cx, cy):
  return Cal3_S2(fx, fy, s, cx, cy)

In [4]:
# Define the camera observation noise model
measurement_noise = gtsam.noiseModel_Isotropic.Sigma(2, 1.0)  # one pixel in u and v

In [5]:
# Create a factor graph
graph = NonlinearFactorGraph()
K = get_camera_calibration(614, 614, 0, 220, 385)

In [6]:
img1 = load_image('img6.jpg', path='../data/lettuce_home/set4/')
img2 = load_image('img8.jpg', path='../data/lettuce_home/set4/')
points_1, points_2, e_estimate, r, t = get_matches_and_e(img1, img2)

selected points with ratio:  10


In [7]:
# add all the image points to the factor graph

for (i, point) in enumerate(points_1):
  # wrap the point in a measurement
  #print('adding point for camera1')
  factor = GenericProjectionFactorCal3_S2(
            Point2(point), measurement_noise, symbol('x', 0), symbol('l', i), K)
  graph.push_back(factor)
  
for (i, point) in enumerate(points_2):
  #print('adding point for camera2')
  factor = GenericProjectionFactorCal3_S2(
            Point2(point), measurement_noise, symbol('x', 1), symbol('l', i), K)
  graph.push_back(factor)

In [8]:
# Add a prior on pose of camera 1.
# 0.3 rad std on roll,pitch,yaw and 0.1m on x,y,z
pose_noise = gtsam.noiseModel_Diagonal.Sigmas(np.array([0.3, 0.3, 0.3, 0.1, 0.1, 0.1]))
factor = PriorFactorPose3(symbol('x', 0), Pose3(Rot3.Rodrigues(0, 0, 0), Point3(0, 0, 0)), pose_noise)
graph.push_back(factor)

In [9]:
point_noise = gtsam.noiseModel_Isotropic.Sigma(3, 0.1)
factor = PriorFactorPoint3(symbol('l', 0), Point3(1,0,0), point_noise)
graph.push_back(factor)
graph.print_('Factor Graph:\n')

In [14]:
# Create the data structure to hold the initial estimate to the solution
initial_estimate = Values()

r_init = Rot3.Rodrigues(0, 0, 0)
t_init = Point3(0, 0, 0)
transformed_pose = Pose3(r_init, t_init)
initial_estimate.insert(symbol('x', 0), transformed_pose)

r_init = Rot3(r[0])
t_init = Point3(t[0], t[1], t[2])
transformed_pose = Pose3(r_init, t_init)
initial_estimate.insert(symbol('x', 1), transformed_pose)

for j in range(points_1.shape[0]):
  initial_estimate.insert(symbol('l', j), Point3(np.random.rand(3)*100))
initial_estimate.print_('Initial Estimates:\n')


In [12]:
# Optimize the graph and print results
params = gtsam.DoglegParams()
params.setVerbosity('TERMINATION')
optimizer = DoglegOptimizer(graph, initial_estimate, params)
print('Optimizing:')

Optimizing:


In [15]:
result = optimizer.optimize()
result.print_('Final results:\n')
print('initial error = {}'.format(graph.error(initial_estimate)))
print('final error = {}'.format(graph.error(result)))

RuntimeError: 
Indeterminant linear system detected while working near variable
7782220156096217132 (Symbol: l44).

Thrown when a linear system is ill-posed.  The most common cause for this
error is having underconstrained variables.  Mathematically, the system is
underdetermined.  See the GTSAM Doxygen documentation at
http://borg.cc.gatech.edu/ on gtsam::IndeterminantLinearSystemException for
more information.